In [1]:
import sys
import os
import math
import random
import heapq 
import time
import copy
import gc
import numpy as np
import pandas as pd
from functools import reduce
from scipy.spatial.distance import pdist
from PIL import Image
import matplotlib.pyplot as plt
import cv2
#import faiss
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.set_device(6)
print (torch.cuda.current_device())

6


In [2]:
#1. Read data with List storage Data:[name,type],I:[img],Y[type]
root_dir = '/data/fjsdata/physionet/MIMIC-CXR' #the path of images
trainset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/CBIR_train.csv" , sep=',')#load dataset
testset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/CBIR_test.csv" , sep=',')#load dataset
tstart = time.time()
#read train image with CV
trData, trI, trY = [],[],[]
for iname, itype in np.array(trainset).tolist():
    try:
        image_path = os.path.join(root_dir, iname)
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        trData.append([iname,itype])
        trI.append(img)
        trY.append(itype)
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(trData),trainset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(trData))
#read test image with CV
teData, teI, teY = [],[],[]
for iname, itype in np.array(testset).tolist():
    try:
        image_path = os.path.join(root_dir, iname)
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        teData.append([iname,itype])
        teI.append(img)
        teY.append(itype)  
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(teData),testset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(teData))
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))

9000 / 9000 The length of train set is 9000
train/p15545849/s14/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15545849/s14/view1_frontal.jpg.jpeg
0 / 900 train/p16333368/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16333368/s01/view1_frontal.jpg.jpeg
0 / 900 train/p12604683/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12604683/s05/view1_frontal.jpg.jpeg
0 / 900 train/p12562737/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12562737/s02/view1_frontal.jpg.jpeg
0 / 900 train/p19285526/s04/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19285526/s04/view1_frontal.jpg.jpeg
0 / 900 train/p10041196/s01/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p10041196/s01/view2_frontal.jpg.jpeg
0 / 900 train/p14854659/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14854659/s01/view1_frontal.jpg.jpeg
0 / 900 train/p11101925/s08/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11101925/s08/vie

0 / 900 train/p14490913/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14490913/s01/view1_frontal.jpg.jpeg
0 / 900 train/p10448584/s02/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p10448584/s02/view2_frontal.jpg.jpeg
0 / 900 train/p19554899/s07/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19554899/s07/view1_frontal.jpg.jpeg
0 / 900 train/p17471483/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17471483/s02/view1_frontal.jpg.jpeg
0 / 900 train/p11387260/s09/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11387260/s09/view1_frontal.jpg.jpeg
0 / 900 train/p14228640/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14228640/s02/view1_frontal.jpg.jpeg
0 / 900 train/p15511142/s15/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15511142/s15/view1_frontal.jpg.jpeg
0 / 900 train/p14587243/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14587243/s01/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p12987308/s18/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12987308/s18/view1_frontal.jpg.jpeg
0 / 900 train/p11304959/s18/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11304959/s18/view1_frontal.jpg.jpeg
0 / 900 train/p15649581/s22/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15649581/s22/view1_frontal.jpg.jpeg
0 / 900 train/p18131667/s17/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18131667/s17/view1_frontal.jpg.jpeg
0 / 900 train/p17396019/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17396019/s01/view1_frontal.jpg.jpeg
0 / 900 train/p17018278/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17018278/s05/view1_frontal.jpg.jpeg
0 / 900 train/p17809557/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17809557/s01/view1_frontal.jpg.jpeg
0 / 900 train/p19783525/s02/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19783525/s02/view2_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p15230838/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15230838/s05/view1_frontal.jpg.jpeg
0 / 900 train/p19506489/s03/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19506489/s03/view2_frontal.jpg.jpeg
0 / 900 train/p13276100/s06/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13276100/s06/view1_frontal.jpg.jpeg
0 / 900 train/p17741087/s25/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17741087/s25/view1_frontal.jpg.jpeg
0 / 900 train/p19410886/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19410886/s01/view1_frontal.jpg.jpeg
0 / 900 train/p15425725/s23/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15425725/s23/view1_frontal.jpg.jpeg
0 / 900 train/p15372450/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15372450/s01/view1_frontal.jpg.jpeg
0 / 900 train/p18152916/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18152916/s01/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p15604729/s01/view3_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15604729/s01/view3_frontal.jpg.jpeg
0 / 900 train/p18806464/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18806464/s01/view1_frontal.jpg.jpeg
0 / 900 train/p10839295/s03/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p10839295/s03/view1_frontal.jpg.jpeg
0 / 900 train/p11173142/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11173142/s02/view1_frontal.jpg.jpeg
0 / 900 train/p16435208/s09/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16435208/s09/view1_frontal.jpg.jpeg
0 / 900 train/p18489225/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18489225/s01/view1_frontal.jpg.jpeg
0 / 900 train/p11132352/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11132352/s01/view1_frontal.jpg.jpeg
0 / 900 train/p15035317/s03/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15035317/s03/view2_frontal.jpg.jpeg
0 / 900 train/p1

...

0 / 900 train/p18048805/s06/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18048805/s06/view1_frontal.jpg.jpeg
0 / 900 train/p13192224/s33/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13192224/s33/view1_frontal.jpg.jpeg
0 / 900 train/p16490533/s06/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16490533/s06/view1_frontal.jpg.jpeg
0 / 900 train/p16370759/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16370759/s01/view1_frontal.jpg.jpeg
0 / 900 train/p15593752/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15593752/s02/view1_frontal.jpg.jpeg
0 / 900 train/p16720509/s10/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16720509/s10/view1_frontal.jpg.jpeg
0 / 900 train/p17054151/s03/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17054151/s03/view1_frontal.jpg.jpeg
0 / 900 train/p13272142/s04/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13272142/s04/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p13709820/s14/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13709820/s14/view1_frontal.jpg.jpeg
0 / 900 train/p19109226/s02/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19109226/s02/view2_frontal.jpg.jpeg
0 / 900 train/p10794237/s01/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p10794237/s01/view2_frontal.jpg.jpeg
0 / 900 train/p14203508/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14203508/s01/view1_frontal.jpg.jpeg
0 / 900 train/p15398539/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15398539/s05/view1_frontal.jpg.jpeg
0 / 900 train/p19242238/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19242238/s01/view1_frontal.jpg.jpeg
0 / 900 train/p19346447/s06/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19346447/s06/view1_frontal.jpg.jpeg
0 / 900 train/p15389247/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p15389247/s01/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p17810664/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17810664/s01/view1_frontal.jpg.jpeg
0 / 900 train/p11747567/s13/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11747567/s13/view1_frontal.jpg.jpeg
0 / 900 train/p12494365/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12494365/s01/view1_frontal.jpg.jpeg
0 / 900 train/p16345504/s14/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p16345504/s14/view1_frontal.jpg.jpeg
0 / 900 train/p19078379/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19078379/s05/view1_frontal.jpg.jpeg
0 / 900 train/p11535847/s03/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11535847/s03/view2_frontal.jpg.jpeg
0 / 900 train/p18925608/s04/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18925608/s04/view1_frontal.jpg.jpeg
0 / 900 train/p14720255/s08/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14720255/s08/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p11804719/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11804719/s02/view1_frontal.jpg.jpeg
0 / 900 train/p11028696/s03/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p11028696/s03/view1_frontal.jpg.jpeg
0 / 900 train/p13438658/s35/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13438658/s35/view1_frontal.jpg.jpeg
0 / 900 train/p17119475/s03/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p17119475/s03/view1_frontal.jpg.jpeg
0 / 900 train/p19826426/s05/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19826426/s05/view1_frontal.jpg.jpeg
0 / 900 train/p13430469/s11/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13430469/s11/view1_frontal.jpg.jpeg
0 / 900 train/p12405648/s19/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12405648/s19/view1_frontal.jpg.jpeg
0 / 900 train/p14791450/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14791450/s01/view1_frontal.jpg.jpeg
0 / 900 train/p1

0 / 900 train/p12393609/s11/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p12393609/s11/view1_frontal.jpg.jpeg
0 / 900 train/p19145704/s02/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19145704/s02/view1_frontal.jpg.jpeg
0 / 900 train/p18234511/s04/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p18234511/s04/view1_frontal.jpg.jpeg
0 / 900 train/p19920828/s03/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p19920828/s03/view1_frontal.jpg.jpeg
0 / 900 train/p13818699/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p13818699/s01/view1_frontal.jpg.jpeg
0 / 900 train/p10073182/s02/view2_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p10073182/s02/view2_frontal.jpg.jpeg
0 / 900 train/p14766230/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14766230/s01/view1_frontal.jpg.jpeg
0 / 900 train/p14370141/s01/view1_frontal.jpg:/data/fjsdata/physionet/MIMIC-CXR/train/p14370141/s01/view1_frontal.jpg.jpeg
0 / 900 train/p1

In [4]:
#2. define Hashing network with pytorch
class HNet(nn.Module): #deep Hashint Network:DHNet
    def __init__(self,inChannels=3):
        super(HNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,512,512)->(8,256,256)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(8,256,256)->(8,128,128)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(8)
        #layer: spatial attention, pass
        #layer3: Convolution, (8,128,128)->(2,64,64)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(2)
        self.relu2 = nn.ReLU(inplace=True)
        #layer4: mean pooling, (2,64,64)->(2,32,32)
        self.avgpool = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(2)
        #layer5: fully connected, 2*32*32->512
        self.fcl1 = nn.Linear(2*32*32,512)
        self.relu3 = nn.ReLU(inplace=True)
        #layer6: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,16)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer: attention,pass
        #layer3: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer4: mean pooling
        x = self.avgpool(x)
        x = self.bn4(x)
        #layer5:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu3(x)
        #layer6: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss

#test network: valid
x1 = torch.rand(10,3,512,512)#.cuda()
x2 = torch.rand(10,3,512,512)#.cuda()
y = torch.FloatTensor([0,1,1,0,1,0,0,0,1,1])#.cuda()
model = HNet()#.cuda()
criterion  = HashLossFunc(margin=0.5)#.cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
for epoch in range(10):
    optimizer.zero_grad()
    
    out1 = model(x1)#out.grad_fn
    out2 = model(x2)
    loss = criterion(out1,out2,y)
    print (loss.item())
    loss.backward()
    optimizer.step()
    #observe the variant of model.parameters
    for i in model.named_parameters():
        print(i[0])
        print(i[1][0][0][0])
        break
#output
x3 = torch.rand(10,3,512,512)#.cuda()
out3 = model(x3)
print (out3)
out3 = torch.sign(out3) #Binarization,[-1,1]->{-1,1}
print (out3)
print (out3.size())
del x1,x2,x3,out1,out2,out3,model
gc.collect()

17.386795043945312
conv1.weight
tensor([-0.1478,  0.1599,  0.1831])
15.438716888427734
conv1.weight
tensor([-0.1473,  0.1602,  0.1839])
10.893563270568848
conv1.weight
tensor([-0.1470,  0.1601,  0.1848])
6.695099830627441
conv1.weight
tensor([-0.1467,  0.1599,  0.1857])
3.871767044067383
conv1.weight
tensor([-0.1463,  0.1597,  0.1866])
2.785606861114502
conv1.weight
tensor([-0.1461,  0.1596,  0.1871])
2.2012667655944824
conv1.weight
tensor([-0.1458,  0.1593,  0.1872])
1.2126400470733643
conv1.weight
tensor([-0.1454,  0.1592,  0.1871])
0.6458429098129272
conv1.weight
tensor([-0.1452,  0.1590,  0.1868])
0.2603705823421478
conv1.weight
tensor([-0.1449,  0.1585,  0.1869])
tensor([[-0.0516,  0.0075,  0.2112,  0.1801, -0.2960,  0.0338, -0.1678,
         -0.0883,  0.0674,  0.2314,  0.2118,  0.2356,  0.1509,  0.1169,
          0.0294,  0.1005],
        [-0.1346, -0.0061, -0.3142,  0.0765,  0.1832, -0.2936,  0.4371,
         -0.0548, -0.0529, -0.0687, -0.1948, -0.1204, -0.4164,  0.3274,
       

0

In [5]:
#3.train and evaluate model
def onlineGenImgPairs(batchSize):
    idx_sf = random.sample(range(0, len(trY)),2*batchSize)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf
        
#define model
model = HNet().cuda()
criterion  = HashLossFunc(margin=0.5).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
best_net, best_loss = None, float('inf')
for epoch in range(2):#iteration: len(trY)/2
    batchSize = 10
    batches = len(trY)//batchSize
    losses = []
    for batch in range(batches):
        #grad vanish
        optimizer.zero_grad() 
        #genenate training images pair
        trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs(batchSize)
        I1_batch = torch.from_numpy(trI1_sf).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf).type(torch.FloatTensor).cuda()
        #forword
        X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(batch+1, batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teY[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 900 / 900 : loss = 19.540699Eopch:     1 mean_loss = 18.554168
 900 / 900 : loss = 18.211918Eopch:     2 mean_loss = 18.407680
best_loss = 18.407680
 89 / 90 0 mHR@5=0.332667, mAP@5=0.319630, mRR@5=0.898743
mHR@10=0.334333, mAP@10=0.310819, mRR@10=0.831966
mHR@15=0.334296, mAP@15=0.304690, mRR@15=0.797310
mHR@20=0.330333, mAP@20=0.295861, mRR@20=0.745261


In [26]:
#generate dataset
dataset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/train.csv" , sep=',')
#print (dataset.columns)
#print (dataset['view'].value_counts())
dataset = dataset[dataset['view']=='frontal']#only get frontal view
dataset = dataset[['path','Pneumonia']]
#print (dataset['Pneumonia'].value_counts())
trainset, testset = [], []
ds =dataset[dataset['Pneumonia']==0.0].sample(3300).sample(frac=1)#26296-3300, negative
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
ds =dataset[dataset['Pneumonia']==1.0].sample(3300).sample(frac=1)#18327-3300, positive
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
ds =dataset[dataset['Pneumonia']==-1.0].sample(3300).sample(frac=1)#19681-3300, positive
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
print (len(trainset))
print (len(testset))
pd.DataFrame(trainset).to_csv('/data/fjsdata/physionet/MIMIC-CXR/CBIR_train.csv',index=False)
pd.DataFrame(testset).to_csv('/data/fjsdata/physionet/MIMIC-CXR/CBIR_test.csv',index=False)
del ds,dataset
gc.collect() #release cpu memory

9000
900


7